Load corpus

In [1]:
import pandas as pd
import numpy as np
import json
import csv

seasons = pd.read_csv("./data/bangumi.csv", delimiter=",", encoding="utf-8")
episodes = pd.read_csv("./data/episode.csv", delimiter=",", encoding="utf-8")
danmaku_complete = pd.read_csv("./data/danmaku_complete.csv", delimiter="\t", encoding="utf-8", quoting=csv.QUOTE_NONE, low_memory=False)
danmaku_complete = danmaku_complete.fillna(-1)
danmaku_complete.shape

(21573874, 8)

word segment

In [2]:
import jieba.posseg as segtool
import re

ACCEPTABLE_TYPE = {'n', 't', 's', 'f', 'v', 'a', 'b', 'z', 'e', 'y', 'o'}
REPLACE_DICT = {
    "233+": "233",
    "666+": "666",
    "emm+": "emm",
    "hhh+": "hhh",
    "www+": "www"
}

def check_type(word_type):
    if word_type[0] in ACCEPTABLE_TYPE:
        return True
    else:
        return False

def check_replace(word):
    for item in REPLACE_DICT.keys():
        pattern = re.compile(item)
        if re.match(pattern, word) is not None:
            new_word = REPLACE_DICT[item]
            return new_word
    return word

def word_segment(content):
    words = []
    results = segtool.cut(content)
    for result in results:
        result.word = check_replace(result.word)
        if check_type(result.flag):
            words.append(result.word)
    return words

dataset definition

In [36]:
import torch
import torch.utils.data as data
import math
import numpy as np
import pandas as pd
import random
import collections

def common_words(content_a, content_b):
    word_set = set(content_a)
    count = 0
    for word in content_b:
        if word in word_set:
            count += 1
    return count


def negative_sampling(sample, sorted_dm_samples):
    content = sample['content']
    raw_id = sample['raw_id']
    result = None
    while result is None:
        candidate = np.random.choice(len(sorted_dm_samples), 1)
        sample_ = sorted_dm_samples[candidate[0]]
        if common_words(content, sample_['content']) == 0:
            result = candidate[0]
    return result
        
    

class DmDataset(data.Dataset):
    def __init__(self, sorted_aggr_samples, context_size, common_words_min_count):
        self.rawId_to_ix = dict()
        self.dm_samples = []
        
        print('building samples...')
        self.samples = []
        for episode_lvl_samples in sorted_aggr_samples:
            context_start_index = 0
            ix = 0
            for sample in episode_lvl_samples:
                self.dm_samples.append(sample)
                self.rawId_to_ix[sample['raw_id']] = ix
                playback_time = sample['playback_time']
                content = sample['content']
            
                # update context_start_index
                start_sample = episode_lvl_samples[context_start_index]
                while start_sample['playback_time'] < playback_time - context_size:
                    context_start_index += 1
                    start_sample = episode_lvl_samples[context_start_index]
                # build sample pair in context
                it = context_start_index
                sample_ = episode_lvl_samples[it]
                while sample_['playback_time'] <= playback_time + context_size:
                    if sample['raw_id'] != sample_['raw_id'] and common_words(content, sample_['content']) >= common_words_min_count:
                        self.samples.append((sample['raw_id'], sample_['raw_id']))
                    it += 1
                    if it >= len(episode_lvl_samples):
                        break
                    else:
                        sample_ = episode_lvl_samples[it]
                ix += 1
        
        self.dm_size = len(self.dm_samples)
        print('%d samples constructed.' % len(self.samples))
        return 

    def __getitem__(self, index):
        sample = self.samples[index]
        target = self.rawId_to_ix[sample[0]]
        context = self.rawId_to_ix[sample[1]]
        neg_sample = negative_sampling(self.dm_samples[target], self.dm_samples)
        sample_dict = {
            'pos_u': target,
            'pos_v': context,
            'neg_v': neg_sample
        }
        return sample_dict
    
    def __len__(self):
        return len(self.samples)


data preprocess

In [38]:
import pickle


danmaku_selected = danmaku_complete[danmaku_complete['season_id']=='24581']
grouped = danmaku_selected.groupby('episode_id')

context_size = 2.5
common_words_min_count = 2
neg_sampling_num = 10
samples = []

for episode_id, group_data in grouped:
    group_data.sort_values(by='playback_time')
    episode_lvl_samples = []
    print('Processing Episode %d' % episode_id)
    for index, row in group_data.iterrows():
        content = row['content']
        words = word_segment(str(content))
        sample = {
            'raw_id': row['tsc_raw_id'],
            'content': words,
            'playback_time': row['playback_time']
        }
        episode_lvl_samples.append(sample)
    samples.append(episode_lvl_samples)

dm_set = DmDataset(samples, context_size, common_words_min_count)

with open('tmp/dm_dataset.p', 'wb') as f:
    pickle.dump(dm_set, f)

Processing Episode 234406
Processing Episode 234407
Processing Episode 234408
Processing Episode 234409
Processing Episode 234410
Processing Episode 234411
Processing Episode 234412
Processing Episode 234413
Processing Episode 234414
building samples...
1799 samples constructed.


Network definition

In [28]:
import torch as t
from torch import nn
from torch.autograd import Variable as V
import torch.nn.functional as F
import pickle

class TSCEmbedLanguageModeler(nn.Module):

    def __init__(self, tsc_size, embedding_dim):
        super(TSCEmbedLanguageModeler, self).__init__()
        self.u_embeddings = nn.Embedding(tsc_size, embedding_dim, sparse=True)   
        self.v_embeddings = nn.Embedding(tsc_size, embedding_dim, sparse=True) 
        self.embedding_dim = embedding_dim
        self.init_emb()
        
    def init_emb(self):
        initrange = 0.5 / self.embedding_dim
        self.u_embeddings.weight.data.uniform_(-initrange, initrange)
        self.v_embeddings.weight.data.uniform_(-0, 0)

    def forward(self, u_pos, v_pos, v_neg, batch_size):
        embed_u = self.u_embeddings(u_pos)
        embed_v = self.v_embeddings(v_pos)
        
        pdist = nn.PairwiseDistance(p=2, eps=1e-06)
        sim_pos = pdist(embed_u, embed_v)
        score_pos = t.exp(sim_pos)
        
        neg_embed_v = self.v_embeddings(v_neg)
        neg_pos = pdist(embed_u, neg_embed_v)
        score_neg = t.exp(neg_pos)
            
        loss = t.log(score_pos / (score_pos + score_neg))

        return -1*loss.sum()/batch_size
    
    def save_emb(self, path, rawid_to_ix):
        embeds = self.u_embeddings.weight.data.cpu()
        embed_dict = dict()
        with open(path, 'wb') as f:
            for raw_id in rawid_to_ix:
                ix = rawid_to_ix[raw_id]
                embed_dict[raw_id] = embeds[ix].numpy()
            pickle.dump(embed_dict, f)
        return
            

training phase

In [29]:
import torch
import torch.utils.data as Data
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline

is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython:
    from IPython import display
    
def plot_durations(y):
    plt.figure(1)
    plt.clf()
    plt.plot(y)

#     plt.pause(0.001)  # pause a bit so that plots are updated
    if is_ipython:
        display.clear_output(wait=True)
        display.display(plt.gcf())

torch.manual_seed(1) 

EMBEDDING_DIM = 200
batch_size=128
epoch_num = 20

dm_dataloader = Data.DataLoader(
    dataset=dm_set,
    batch_size=128,
    shuffle=True,
    drop_last=True,
    num_workers=8
)

model = TSCEmbedLanguageModeler(dm_set.dm_size, EMBEDDING_DIM)
print(model)
if torch.cuda.is_available():
    model.cuda()
optimizer = optim.SGD(model.parameters(), lr=0.3)

for epoch in range(epoch_num):
    for batch_idx, sample in enumerate(dm_dataloader):
        pos_u = Variable(torch.LongTensor(sample['pos_u']))
        pos_v = Variable(torch.LongTensor(sample['pos_v']))
        neg_v = Variable(torch.LongTensor(sample['neg_v']))
        
        if torch.cuda.is_available():
            pos_u = pos_u.cuda()
            pos_v = pos_v.cuda()
            neg_v = neg_v.cuda()
            
        optimizer.zero_grad()
        loss = model(pos_u, pos_v, neg_v, batch_size)
        print('epoch: %d batch %d : loss: %4.4f' % (epoch, batch_idx, loss.data[0]))
        
        loss.backward()
        
        optimizer.step()
        

TSCEmbedLanguageModeler (
  (u_embeddings): Embedding(32085, 200, sparse=True)
  (v_embeddings): Embedding(32085, 200, sparse=True)
)
epoch: 0 batch 0 : loss: 0.6931
epoch: 0 batch 1 : loss: 0.6932
epoch: 0 batch 2 : loss: 0.6932
epoch: 0 batch 3 : loss: 0.6932
epoch: 0 batch 4 : loss: 0.6932
epoch: 0 batch 5 : loss: 0.6932
epoch: 0 batch 6 : loss: 0.6932
epoch: 0 batch 7 : loss: 0.6932
epoch: 0 batch 8 : loss: 0.6931
epoch: 0 batch 9 : loss: 0.6932
epoch: 0 batch 10 : loss: 0.6932
epoch: 0 batch 11 : loss: 0.6932
epoch: 0 batch 12 : loss: 0.6932
epoch: 0 batch 13 : loss: 0.6932
epoch: 0 batch 14 : loss: 0.6931
epoch: 0 batch 15 : loss: 0.6932
epoch: 0 batch 16 : loss: 0.6932
epoch: 0 batch 17 : loss: 0.6932
epoch: 0 batch 18 : loss: 0.6932
epoch: 0 batch 19 : loss: 0.6932
epoch: 0 batch 20 : loss: 0.6932
epoch: 0 batch 21 : loss: 0.6932
epoch: 0 batch 22 : loss: 0.6931
epoch: 0 batch 23 : loss: 0.6931
epoch: 0 batch 24 : loss: 0.6932
epoch: 0 batch 25 : loss: 0.6932
epoch: 0 batch 26 

epoch: 0 batch 291 : loss: 0.6922
epoch: 0 batch 292 : loss: 0.6922
epoch: 0 batch 293 : loss: 0.6920
epoch: 0 batch 294 : loss: 0.6918
epoch: 0 batch 295 : loss: 0.6921
epoch: 0 batch 296 : loss: 0.6920
epoch: 0 batch 297 : loss: 0.6916
epoch: 0 batch 298 : loss: 0.6921
epoch: 0 batch 299 : loss: 0.6917
epoch: 0 batch 300 : loss: 0.6921
epoch: 0 batch 301 : loss: 0.6918
epoch: 0 batch 302 : loss: 0.6921
epoch: 0 batch 303 : loss: 0.6919
epoch: 0 batch 304 : loss: 0.6917
epoch: 0 batch 305 : loss: 0.6919
epoch: 0 batch 306 : loss: 0.6920
epoch: 0 batch 307 : loss: 0.6918
epoch: 0 batch 308 : loss: 0.6918
epoch: 0 batch 309 : loss: 0.6916
epoch: 0 batch 310 : loss: 0.6919
epoch: 0 batch 311 : loss: 0.6917
epoch: 0 batch 312 : loss: 0.6916
epoch: 0 batch 313 : loss: 0.6920
epoch: 0 batch 314 : loss: 0.6918
epoch: 0 batch 315 : loss: 0.6918
epoch: 0 batch 316 : loss: 0.6916
epoch: 0 batch 317 : loss: 0.6918
epoch: 0 batch 318 : loss: 0.6916
epoch: 0 batch 319 : loss: 0.6920
epoch: 0 batch

epoch: 0 batch 628 : loss: 0.6857
epoch: 0 batch 629 : loss: 0.6856
epoch: 0 batch 630 : loss: 0.6852
epoch: 0 batch 631 : loss: 0.6865
epoch: 0 batch 632 : loss: 0.6861
epoch: 0 batch 633 : loss: 0.6842
epoch: 0 batch 634 : loss: 0.6835
epoch: 0 batch 635 : loss: 0.6851
epoch: 0 batch 636 : loss: 0.6841
epoch: 0 batch 637 : loss: 0.6854
epoch: 0 batch 638 : loss: 0.6865
epoch: 0 batch 639 : loss: 0.6869
epoch: 0 batch 640 : loss: 0.6861
epoch: 0 batch 641 : loss: 0.6844
epoch: 0 batch 642 : loss: 0.6858
epoch: 0 batch 643 : loss: 0.6846
epoch: 0 batch 644 : loss: 0.6864
epoch: 0 batch 645 : loss: 0.6856
epoch: 0 batch 646 : loss: 0.6842
epoch: 0 batch 647 : loss: 0.6854
epoch: 0 batch 648 : loss: 0.6844
epoch: 0 batch 649 : loss: 0.6853
epoch: 0 batch 650 : loss: 0.6846
epoch: 0 batch 651 : loss: 0.6851
epoch: 0 batch 652 : loss: 0.6857
epoch: 0 batch 653 : loss: 0.6847
epoch: 0 batch 654 : loss: 0.6846
epoch: 0 batch 655 : loss: 0.6838
epoch: 0 batch 656 : loss: 0.6851
epoch: 0 batch

epoch: 0 batch 957 : loss: 0.6773
epoch: 0 batch 958 : loss: 0.6800
epoch: 0 batch 959 : loss: 0.6806
epoch: 0 batch 960 : loss: 0.6781
epoch: 0 batch 961 : loss: 0.6794
epoch: 0 batch 962 : loss: 0.6816
epoch: 0 batch 963 : loss: 0.6770
epoch: 0 batch 964 : loss: 0.6745
epoch: 0 batch 965 : loss: 0.6757
epoch: 0 batch 966 : loss: 0.6785
epoch: 0 batch 967 : loss: 0.6790
epoch: 0 batch 968 : loss: 0.6770
epoch: 0 batch 969 : loss: 0.6778
epoch: 0 batch 970 : loss: 0.6777
epoch: 0 batch 971 : loss: 0.6808
epoch: 0 batch 972 : loss: 0.6776
epoch: 0 batch 973 : loss: 0.6778
epoch: 0 batch 974 : loss: 0.6774
epoch: 0 batch 975 : loss: 0.6775
epoch: 0 batch 976 : loss: 0.6787
epoch: 0 batch 977 : loss: 0.6784
epoch: 0 batch 978 : loss: 0.6757
epoch: 0 batch 979 : loss: 0.6762
epoch: 0 batch 980 : loss: 0.6761
epoch: 0 batch 981 : loss: 0.6791
epoch: 0 batch 982 : loss: 0.6775
epoch: 0 batch 983 : loss: 0.6755
epoch: 0 batch 984 : loss: 0.6781
epoch: 0 batch 985 : loss: 0.6780
epoch: 0 batch

epoch: 1 batch 0 : loss: 0.6694
epoch: 1 batch 1 : loss: 0.6753
epoch: 1 batch 2 : loss: 0.6748
epoch: 1 batch 3 : loss: 0.6742
epoch: 1 batch 4 : loss: 0.6732
epoch: 1 batch 5 : loss: 0.6733
epoch: 1 batch 6 : loss: 0.6704
epoch: 1 batch 7 : loss: 0.6681
epoch: 1 batch 8 : loss: 0.6725
epoch: 1 batch 9 : loss: 0.6702
epoch: 1 batch 10 : loss: 0.6708
epoch: 1 batch 11 : loss: 0.6731
epoch: 1 batch 12 : loss: 0.6728
epoch: 1 batch 13 : loss: 0.6739
epoch: 1 batch 14 : loss: 0.6736
epoch: 1 batch 15 : loss: 0.6739
epoch: 1 batch 16 : loss: 0.6716
epoch: 1 batch 17 : loss: 0.6740
epoch: 1 batch 18 : loss: 0.6712
epoch: 1 batch 19 : loss: 0.6659
epoch: 1 batch 20 : loss: 0.6762
epoch: 1 batch 21 : loss: 0.6710
epoch: 1 batch 22 : loss: 0.6725
epoch: 1 batch 23 : loss: 0.6711
epoch: 1 batch 24 : loss: 0.6719
epoch: 1 batch 25 : loss: 0.6727
epoch: 1 batch 26 : loss: 0.6697
epoch: 1 batch 27 : loss: 0.6738
epoch: 1 batch 28 : loss: 0.6683
epoch: 1 batch 29 : loss: 0.6714
epoch: 1 batch 30 : 

...

epoch: 8 batch 1201 : loss: 0.5139
epoch: 8 batch 1202 : loss: 0.5232
epoch: 8 batch 1203 : loss: 0.5259
epoch: 8 batch 1204 : loss: 0.5020
epoch: 8 batch 1205 : loss: 0.5263
epoch: 8 batch 1206 : loss: 0.5225
epoch: 8 batch 1207 : loss: 0.5291
epoch: 8 batch 1208 : loss: 0.5200
epoch: 8 batch 1209 : loss: 0.5234
epoch: 8 batch 1210 : loss: 0.5306
epoch: 8 batch 1211 : loss: 0.4990
epoch: 8 batch 1212 : loss: 0.5249
epoch: 8 batch 1213 : loss: 0.5171
epoch: 8 batch 1214 : loss: 0.5440
epoch: 8 batch 1215 : loss: 0.4993
epoch: 8 batch 1216 : loss: 0.5242
epoch: 8 batch 1217 : loss: 0.5124
epoch: 8 batch 1218 : loss: 0.5117
epoch: 8 batch 1219 : loss: 0.5404
epoch: 8 batch 1220 : loss: 0.5292
epoch: 8 batch 1221 : loss: 0.5223
epoch: 8 batch 1222 : loss: 0.5125
epoch: 8 batch 1223 : loss: 0.5305
epoch: 8 batch 1224 : loss: 0.5588
epoch: 8 batch 1225 : loss: 0.5370
epoch: 8 batch 1226 : loss: 0.5345
epoch: 8 batch 1227 : loss: 0.5095
epoch: 8 batch 1228 : loss: 0.4981
epoch: 8 batch 1229 

epoch: 9 batch 219 : loss: 0.4988
epoch: 9 batch 220 : loss: 0.5047
epoch: 9 batch 221 : loss: 0.5170
epoch: 9 batch 222 : loss: 0.5109
epoch: 9 batch 223 : loss: 0.5350
epoch: 9 batch 224 : loss: 0.5285
epoch: 9 batch 225 : loss: 0.5168
epoch: 9 batch 226 : loss: 0.5137
epoch: 9 batch 227 : loss: 0.5267
epoch: 9 batch 228 : loss: 0.5004
epoch: 9 batch 229 : loss: 0.5341
epoch: 9 batch 230 : loss: 0.5031
epoch: 9 batch 231 : loss: 0.5038
epoch: 9 batch 232 : loss: 0.5036
epoch: 9 batch 233 : loss: 0.5121
epoch: 9 batch 234 : loss: 0.5014
epoch: 9 batch 235 : loss: 0.5009
epoch: 9 batch 236 : loss: 0.5022
epoch: 9 batch 237 : loss: 0.4988
epoch: 9 batch 238 : loss: 0.5264
epoch: 9 batch 239 : loss: 0.5486
epoch: 9 batch 240 : loss: 0.5022
epoch: 9 batch 241 : loss: 0.4900
epoch: 9 batch 242 : loss: 0.5092
epoch: 9 batch 243 : loss: 0.5180
epoch: 9 batch 244 : loss: 0.5247
epoch: 9 batch 245 : loss: 0.4970
epoch: 9 batch 246 : loss: 0.4834
epoch: 9 batch 247 : loss: 0.5253
epoch: 9 batch

epoch: 9 batch 554 : loss: 0.5154
epoch: 9 batch 555 : loss: 0.5058
epoch: 9 batch 556 : loss: 0.5080
epoch: 9 batch 557 : loss: 0.5011
epoch: 9 batch 558 : loss: 0.5048
epoch: 9 batch 559 : loss: 0.5024
epoch: 9 batch 560 : loss: 0.5046
epoch: 9 batch 561 : loss: 0.5015
epoch: 9 batch 562 : loss: 0.5192
epoch: 9 batch 563 : loss: 0.5244
epoch: 9 batch 564 : loss: 0.5158
epoch: 9 batch 565 : loss: 0.4991
epoch: 9 batch 566 : loss: 0.5133
epoch: 9 batch 567 : loss: 0.5088
epoch: 9 batch 568 : loss: 0.5025
epoch: 9 batch 569 : loss: 0.5178
epoch: 9 batch 570 : loss: 0.4973
epoch: 9 batch 571 : loss: 0.5242
epoch: 9 batch 572 : loss: 0.5010
epoch: 9 batch 573 : loss: 0.4770
epoch: 9 batch 574 : loss: 0.5002
epoch: 9 batch 575 : loss: 0.5052
epoch: 9 batch 576 : loss: 0.5051
epoch: 9 batch 577 : loss: 0.4987
epoch: 9 batch 578 : loss: 0.5079
epoch: 9 batch 579 : loss: 0.5242
epoch: 9 batch 580 : loss: 0.5234
epoch: 9 batch 581 : loss: 0.5011
epoch: 9 batch 582 : loss: 0.4986
epoch: 9 batch

epoch: 9 batch 894 : loss: 0.5212
epoch: 9 batch 895 : loss: 0.5044
epoch: 9 batch 896 : loss: 0.4948
epoch: 9 batch 897 : loss: 0.4931
epoch: 9 batch 898 : loss: 0.5183
epoch: 9 batch 899 : loss: 0.5245
epoch: 9 batch 900 : loss: 0.5158
epoch: 9 batch 901 : loss: 0.4936
epoch: 9 batch 902 : loss: 0.5118
epoch: 9 batch 903 : loss: 0.5013
epoch: 9 batch 904 : loss: 0.5077
epoch: 9 batch 905 : loss: 0.5139
epoch: 9 batch 906 : loss: 0.5197
epoch: 9 batch 907 : loss: 0.4963
epoch: 9 batch 908 : loss: 0.5077
epoch: 9 batch 909 : loss: 0.4955
epoch: 9 batch 910 : loss: 0.5078
epoch: 9 batch 911 : loss: 0.4942
epoch: 9 batch 912 : loss: 0.4806
epoch: 9 batch 913 : loss: 0.5149
epoch: 9 batch 914 : loss: 0.5183
epoch: 9 batch 915 : loss: 0.5155
epoch: 9 batch 916 : loss: 0.5060
epoch: 9 batch 917 : loss: 0.5058
epoch: 9 batch 918 : loss: 0.5107
epoch: 9 batch 919 : loss: 0.4988
epoch: 9 batch 920 : loss: 0.5078
epoch: 9 batch 921 : loss: 0.5082
epoch: 9 batch 922 : loss: 0.4812
epoch: 9 batch

epoch: 9 batch 1228 : loss: 0.5224
epoch: 9 batch 1229 : loss: 0.5059
epoch: 9 batch 1230 : loss: 0.5010
epoch: 9 batch 1231 : loss: 0.5187
epoch: 9 batch 1232 : loss: 0.5260
epoch: 9 batch 1233 : loss: 0.5156
epoch: 9 batch 1234 : loss: 0.5168
epoch: 9 batch 1235 : loss: 0.5083
epoch: 9 batch 1236 : loss: 0.5202
epoch: 9 batch 1237 : loss: 0.5046
epoch: 9 batch 1238 : loss: 0.5249
epoch: 9 batch 1239 : loss: 0.5106
epoch: 9 batch 1240 : loss: 0.5046
epoch: 9 batch 1241 : loss: 0.5067
epoch: 9 batch 1242 : loss: 0.5314
epoch: 9 batch 1243 : loss: 0.4957
epoch: 9 batch 1244 : loss: 0.5163
epoch: 9 batch 1245 : loss: 0.4954
epoch: 9 batch 1246 : loss: 0.4814
epoch: 9 batch 1247 : loss: 0.4956
epoch: 9 batch 1248 : loss: 0.4994
epoch: 9 batch 1249 : loss: 0.5265
epoch: 9 batch 1250 : loss: 0.5189
epoch: 9 batch 1251 : loss: 0.5100
epoch: 9 batch 1252 : loss: 0.5167
epoch: 9 batch 1253 : loss: 0.4948
epoch: 9 batch 1254 : loss: 0.5049
epoch: 9 batch 1255 : loss: 0.5181


In [30]:
model.save_emb('models/dm_embeds.p', dm_set.rawId_to_ix)
with open('models/dm_embeds.p', 'rb') as f:
    embed_dict = pickle.load(f)
len(embed_dict)

32085